# Projeto Spark

Entrega: 31 de maio de 2023

## Introdução

Neste projeto vamos construir um classificador Naive-Bayes para determinar o sentimento de um comentário.

## Grupos

O projeto pode ser individual ou em duplas. Criem os grupos em https://classroom.github.com/a/EdbpYF9x

## Instalando o ambiente

O jeito mais simples de começar a trabalhar com Spark é instalar um container com tudo pronto! No site https://hub.docker.com/r/jupyter/pyspark-notebook vemos uma imagem Docker que já vem com `pyspark` e `jupyter lab`. Instale a imagem com o comando:

```bash
docker pull jupyter/pyspark-notebook
```

Vamos iniciar o ambiente de trabalho com o comando `docker run`. Para isso precisamos tomar alguns cuidados:

1) Temos que mapear nosso diretorio local de trabalho para um diretório interno do container, de modo que alterações feitas dentro do container (nesta pasta escolhida) sejam gravadas no nosso diretorio local. No container temos um usuário padrão com *username* `jovyan`. No *homedir* desse usuario temos uma pasta vazia `work`, que vai servir como local de mapeamento do nosso diretorio local de trabalho. Podemos então fazer esse mapeamendo com a opção `-v` do comando `docker run` da seguinte forma:

```bash
-v <diretorio>:/home/jovyan/work
```

onde `<diretorio>` representa seu diretorio local de trabalho.

2) Para acessar o `jupyter notebook` e o *dashboard* do Spark a partir do nosso *browser* favorito temos que abrir algumas portas do container com a opção `-p`. As portas são `8888` (para o próprio `jupyter notebook`) e `4040` (para o *dashboard* do Spark). Ou seja, adicionaremos às opções do `docker run`o seguinte:

```bash
-p 8888:8888 -p 4040:4040
```

Desta forma, ao acessar `localhost:8888` na nossa máquina, estaremos acessando o servidor Jupyter na porta 8888 interna do container.

3) Vamos iniciar o container no modo interativo, e vamos especificar que o container deve ser encerrado ao fechar o servidor Jupyter. Faremos isso com as opções `-it` e `-rm`

Portanto, o comando completo que eu uso na minha máquina Linux para iniciar o container é:

```bash
docker run \
    -it \
    --rm \
    -p 8888:8888 \
    -p 4040:4040 \
    -v "`pwd`":/home/jovyan/work \
    jupyter/pyspark-notebook
```

Para facilitar a vida eu coloco esse comando em um arquivo `inicia.sh`. Engenheiros, façam do jeito que preferirem!

Agora abra esse notebook lá no container!


## Iniciando o Spark

Vamos iniciar o ambiente Spark. Para isso vamos:

1) Criar um objeto de configuração do ambiente Spark. Nossa configuração será simples: vamos especificar que o nome da nossa aplicação Spark é "Minha aplicação", e que o *master node* é a máquina local, usando todos os *cores* disponíveis. Aplicações reais de Spark são configuradas de modo ligeiramente diferente: ao especificar o *master node* passamos uma URL real, com o endereço do nó gerente do *cluster* Spark.

2) Vamos criar um objeto do tipo `SparkContext` com essa configuração

In [ ]:
import pyspark

conf = pyspark.SparkConf()
conf.setAppName('Minha aplicação')
conf.setMaster('local[*]')

sc = pyspark.SparkContext(conf=conf)

O `SparkContext` é a nossa porta de entrada para o cluster Spark, ele será a raiz de todas as nossas operações com o Spark.

In [ ]:
sc

O link acima provavelmente não funcionará porque ele se refere à porta 4040 interna do container (portanto a URL está com endereço interno). Porém fizemos o mapeamento da porta 4040 interna para a porta 4040 externa, logo você pode acessar o *dashboard* do Spark no endereço http://localhost:4040

<center><img src="./spark_dashboard.png" width=800/></center>

## Lendo os dados

Vamos começar lendo o arquivo de reviews e gravando o resultado em formato pickle, mais amigável.

In [ ]:
def parse_line(line):
    parts = line[1:-1].split('","')
    sentiment = int(parts[0])
    title = parts[1].replace('""', '"')
    body = parts[2].replace('""', '"')
    return (sentiment, title, body)

rdd = sc.textFile('data/train.csv').map(parse_line)

In [ ]:
rdd.count()

In [ ]:
rdd.take(1)

Agora vamos gravar no formato pickle, para facilitar os trabalhos futuros. Após gravar o arquivo, não mais rode as células desta primeira etapa!

In [ ]:
rdd.saveAsPickleFile('reviews.pickle')

## Um classificador Naive-Bayes

In [2]:
import string
import math
import pyspark

conf = pyspark.SparkConf()
conf.setAppName('Minha aplicação')
conf.setMaster('local[*]')

sc = pyspark.SparkContext(conf=conf)

Vamos ler o arquivo pickle gravado anteriormente:

In [3]:
rdd = sc.pickleFile('reviews.pickle')

In [4]:
rdd.count()

3600000

In [5]:
rdd.take(1)

[(2,
  'Stuning even for the non-gamer',
  'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^')]

Agora, complete as tarefas em sequencia para construir o classificador Naive-Bayes:

### Fase 1

#### Tarefa

Construa uma função que recebe um RDD no formato do RDD original e retorna um RDD no qual cada item é um par (palavra, contagem).

In [6]:
def separa_em_palavras(item):
    titulo = item[1]
    corpo = item[2]
    return titulo.split() + corpo.split()

def sem_pontuacao(x):
    for c in x:
        if c in string.punctuation:
            return False
    return True


def contagem(rdd_original):
    rdd_split = rdd_original \
        .flatMap(separa_em_palavras) \
        .filter(sem_pontuacao)\
        .map(lambda x: (x, 1)) \
        .reduceByKey(lambda x, y: x + y)
    return rdd_split

In [ ]:
rdd_split = contagem(rdd)
rdd_split.takeOrdered(10, lambda x: -x[1])

#### Tarefa

Construa uma função que recebe o RDD (palavra, contagem) construido anteriormente e retorna um RDD no qual cada item é um par (palavra, $\log_{10}\left(c \, / \, T\right)$), onde $c$ é a contagem daquela palavra e $T$ é a soma das contagens de palavra.

In [7]:
def contagem_total(rdd_original):
    contagem = rdd_original \
                .flatMap(separa_em_palavras) \
                .filter(sem_pontuacao)\
                .count()
    return contagem

In [8]:
T_original = contagem_total(rdd)

In [9]:
def log_contagem(rdd_split, T):
    rdd_log = rdd_split \
                .map(lambda x: (x[0], math.log(x[1]/T, 10)))
    return rdd_log

In [ ]:
rdd_log = log_contagem(rdd_split, T_original)
rdd_log.takeOrdered(10, lambda x: -x[1])

#### Tarefa

Separe o RDD original em dois RDDs: o dos reviews positivos e o dos negativos. Em seguida, use as funções anteriores para construir RDDs que contem os pares (palavra, $\log_{10}\left(c \, / \, T\right)$)

In [10]:
rdd_positivos = rdd.filter(lambda x: x[0] == 2)
rdd_negativos = rdd.filter(lambda x: x[0] == 1)

In [11]:
rdd_positivos_split = contagem(rdd_positivos)
T_positivos = contagem_total(rdd_positivos)
rdd_positivos_log = log_contagem(rdd_positivos_split, T_positivos)

rdd_negativos_split = contagem(rdd_negativos)
T_negativos = contagem_total(rdd_negativos)
rdd_negativos_log = log_contagem(rdd_negativos_split, T_negativos)

In [12]:
rdd_positivos_log.takeOrdered(10, lambda x: -x[1])

[('the', -1.3087279254776858),
 ('and', -1.4742870907750345),
 ('a', -1.5427896991860932),
 ('to', -1.5616817902847964),
 ('I', -1.5786329779030477),
 ('of', -1.6190216986647599),
 ('is', -1.670517375538686),
 ('this', -1.8268730410869245),
 ('it', -1.8354341370011509),
 ('in', -1.8720079472596638)]

In [43]:
rdd_negativos_log.takeOrdered(10, lambda x: -x[1])

[('the', -1.2898080994079197),
 ('I', -1.5249296734506503),
 ('to', -1.5487266713273227),
 ('and', -1.5620264300551518),
 ('a', -1.5707942641431285),
 ('of', -1.6602686209170103),
 ('is', -1.7355192392404242),
 ('it', -1.7689899474700645),
 ('this', -1.788560315647327),
 ('in', -1.9274169739438267)]

### Tarefa

Use o `.fullOuterJoin()` dos RDDs para construir um RDD unificado, no qual cada item é da forma (palavra, log_prob_positivo, log_prob_negativo). "Baixe" esse resultado final usando `.collect()`.

In [14]:
rdd_outer_joined = rdd_positivos_log.fullOuterJoin(rdd_negativos_log).collect()

In [38]:
rdd_outer_joined[:10]

[('out', (-2.5940007500305606, -2.5276951908083563)),
 ('family', (-3.4122506213707133, -3.852019458860614)),
 ('start', (-3.5014041870006154, -3.5939165435118623)),
 ('willing', (-4.398161900200943, -4.293322053616597)),
 ('Electronics', (-5.777439375734934, -5.4831774397339235)),
 ('gift', (-3.546621895036084, -3.847410736303814)),
 ('southside', (-6.722818424842954, -7.098074655767058)),
 ('guest', (-4.683404305666817, -4.913099465068797)),
 ('cute', (-3.987401002930166, -4.13306120549481)),
 ('JMM', (-7.588119850945497, None))]

#### Tarefa

Para uma dada string, determine se ela é um review positivo ou negativo usando os RDDs acima. Lembre-se de como funciona o classificador Naive-Bayes: http://stanford.edu/~jurafsky/slp3/slides/7_NB.pdf, consulte tambem suas notas de aula de Ciência dos Dados!

In [17]:
prob_priori_frase_positiva = T_positivos/T_original
prob_priori_frase_negativa = T_negativos/T_original

log_prob_priori_frase_positiva = math.log(prob_frase_positiva, 10)
log_prob_priori_frase_negativa = math.log(prob_frase_negativa, 10)

In [45]:
probs_palavras = {}

for e in rdd_outer_joined:
    # probs_palavras[palavra] = (prob_positiva, prob_negativa)
    probs_palavras[e[0]] = (e[1][0], e[1][1])

In [46]:
def naive_bayes(frase, p_P, p_N, p_Palavras):
    prob_frase_positiva = p_P
    prob_frase_negativa = p_N
    
    for palavra in frase.split():
        palavra = ''.join(filter(lambda x: sem_pontuacao(x), palavra))
        prob_frase_positiva += p_Palavras[palavra][0]
        prob_frase_negativa += p_Palavras[palavra][1]
        
    if prob_frase_positiva > prob_frase_negativa:
        return 2
    return 1        

In [51]:
sent = naive_bayes("The sky is so blue i love my life", log_prob_priori_frase_negativa, log_prob_priori_frase_negativa, probs_palavras)
if sent == 2:
    print("Positivo! =)")
elif sent == 1:
    print("Negativo! =(")

Positivo! =)


### Fase 2

Agora que temos um classificador Naive-Bayes, vamos explorá-lo um pouco:

### Tarefa

Quais são as 100 palavras que mais indicam negatividade, ou seja, onde a diferença entre a probabilidade da palavra no conjunto dos comentários negativos e positivos é máxima? E quais as 100 palavras de maior positividade? Mostre os resultados na forma de *word clouds*.

In [58]:
diferencas_negativo = []
diferencas_positivo = []
for e in rdd_outer_joined:
    try:
        diferencas_negativo.append((e[0], (e[1][1]-e[1][0])))
    except TypeError:
        diferencas_negativo.append((e[0], None))
    try:
        diferencas_positivo.append((e[0], (e[1][0]-e[1][1])))
    except TypeError:
        diferencas_positivo.append((e[0], None))

In [59]:
diferencas_negativo.sort(key = lambda x: x[1])
diferencas_positivo.sort(key = lambda x: x[1])

TypeError: '<' not supported between instances of 'NoneType' and 'float'

### Tarefa desafio!

Qual o desempenho do classificador (acurácia)? Para medir sua acurácia:

- Separe os reviews em dois conjuntos: treinamente e teste
- Repita o "treinamento" do classificador com o conjunto de treinamento
- Para cada review do conjunto de teste, determine se é positiva ou negativa de acordo com o classificador
- Determine a acurácia

Esta não é uma tarefa trivial. Não basta fazer um `for` para determinar a classe de cada review de teste: isso demoraria uma eternidade. Você tem que usar variáveis "broadcast" do Spark para enviar uma cópia da tabela de frequencias para cada *core* do executor.

### Tarefa desafio!

Implemente Laplace smoothing

## Rubrica de avaliação

- I: groselha, falha crítica, ou não entregou nada
- D: Fez uma tentativa honesta de fazer todos os itens da fase 1, mas tem erros
- C: Fase 1 completa
- B: Fase 2, faltando apenas um desafio
- A: Fase 2 completa